In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#get our training and test data
train = pd.read_csv('sign_mnist_train.csv')
test = pd.read_csv('sign_mnist_test.csv')

In [ ]:

train.head()


In [ ]:
#get our training labels
labels=train['label'].values

In [ ]:
#view unique labels
unique_val = np.array(labels)
np.unique(unique_val)

In [ ]:
#plot the quantities in each class
plt.figure(figsize=(18,8))
sns.countplot(x=labels)

In [ ]:
train.drop('label',axis=1,inplace=True)

In [ ]:
#extract the image data from each row in our csv , remember its in a row of 784 columns
images=train.values
images=np.array([np.reshape(i,(28,28)) for i in images])
images = np.array([i.flatten() for i in images])

In [ ]:
#hot one encode our labels
from sklearn.preprocessing import LabelBinarizer

# Initialize an instance of LabelBinarizer
label_binarizer = LabelBinarizer()

# Fit and transform the labels
labels = label_binarizer.fit_transform(labels)

In [ ]:
#view labels
labels


In [ ]:
#inspect an image
index=2
print(labels[index])
plt.imshow(images[index].reshape(28,28))

In [ ]:
import cv2
import numpy as np
for i in range(0,10):
    rand=np.random.randint(0,len(images))
    input_im=images[rand]

    sample=input_im.reshape(28,28).astype(np.uint8)
    sample=cv2.resize(sample,None,fx=10,fy=10,interpolation=cv2.INTER_CUBIC)
    cv2.imshow("sample image",sample)
    cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#split our data into xtrain,xtest,ytrain,ytest
from sklearn.model_selection import train_test_split

x_train,x_test,y_train, y_test = train_test_split(images,labels,test_size=0.3,random_state=101)

In [ ]:
#start loading our tensorflow modules and definer oru batch size etc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D ,MaxPooling2D,Flatten,Dropout

batch_size = 128
num_classes = 24
epochs=10

In [ ]:
#scale our images
x_train = x_train / 255
x_test = x_test / 255

In [ ]:
#reshape them into size required by Tf and keras
x_train=x_train.reshape(x_train.shape[0],28,28,1)
x_test=x_test.reshape(x_test.shape[0],28,28,1)

plt.imshow(x_train[0].reshape(28,28))

In [ ]:
# Create our CNN Model

from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam


num_classes=24
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu', input_shape=(28, 28,1)))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))


model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))


model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.20))


model.add(Dense(num_classes, activation = 'softmax'))

In [ ]:
#compile our model
model.compile(loss='categorical_crossentropy',
              optimizer= Adam(),
              metrics=['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
history = model.fit(x_train, y_train, validation_data = (x_test,y_test), epochs=epochs, batch_size=batch_size)

In [ ]:
# Save the model in the native Keras format
model.save("sign_mnist_cnn_50_Epochs.keras.h5")

print("Model Saved")

In [ ]:
# View our training history graphically

plt.plot(history.history['accuracy'])

plt.plot(history.history['val_accuracy'])

plt.title("Accuracy")

plt.xlabel('epoch')

plt.ylabel('accuracy')

plt.legend(['train', 'test'])

plt.show()

In [ ]:
# Reshape our test data so that we can evaluate its performance on unseen data
test_labels = test['label']
test.drop('label', axis=1, inplace=True)

test_images = test.values
test_images = np.array([np.reshape(i, (28, 28)) for i in test_images])
test_images = np.array([i.flatten() for i in test_images])

test_labels = label_binarizer.fit_transform(test_labels)

test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

test_images.shape

y_pred = model.predict(test_images)


In [ ]:

#get our accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(test_labels,y_pred.round())

In [ ]:
# Create function to match Label to letter
def getLetter(result):
    classLabels = {0: 'A',
                   1: 'B',
                   2: 'C',
                   3: 'D',
                   4: 'E',
                   5: 'F',
                   6: 'G',
                   7: 'H',
                   8: 'I',
                   9: 'K',
                   10: 'L',
                   11: 'M',
                   12: 'N',
                   13: '0',
                   14: 'P',
                   15: 'Q',
                   16: 'R',
                   17: '5',
                   18: 'T',
                   19: 'U',
                   20: 'V',
                   21: 'W',
                   22: 'X',
                   23: 'Y'}

    try:
        res = int(result)
        return classLabels[res]
    except:
        return "Error"

In [ ]:
#TEST ON ACTUAL WEBCAM INPUT

In [ ]:
#TEST ON ACTUAL WEBCAM INPUT
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    # Define region of interest
    roi = frame[100:400, 320:620]
    cv2.imshow('roi', roi)
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)

    cv2.imshow('roi scaled and gray', roi)
    copy = frame.copy()
    cv2.rectangle(copy, (320, 100), (620, 400), (255, 0, 0), 5)

    roi = roi.reshape(1, 28, 28, 1)

    result = str(model.predict_classes(roi, 1,  verbose = 0)[0])
    cv2.putText(copy, getLetter(result), (300, 100), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 255, 0), 2)
    cv2.imshow('frame', copy)

    if cv2.waitKey(1) == 13:
        break

cap.release()
cv2.destroyAllWindows()